In [ ]:
import math
from scipy import stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# **Question Formualation & Data Analysis**

## **Q1: Các yếu tố về gia đình (như `famsize`, `Medu`, `Fedu`, ...) có ảnh hưởng đến kết quả học tập của một học sinh hay không?**

### **The question**
- Tập trung vào mối quan hệ giữa các biến độc lập thuộc nhóm gia đình (`famsize`, `Pstatus`, `Medu`,...) và biến Target là điểm số cuối kì `G3`

### **Motivation and Benefits**
**Why is this question worth investigating?**

Gia đình là môi trường giáo dục đầu tiên, có ảnh hưởng lớn để sự phát triển của trẻ vị thành niên. Việc xác định những yếu tố về gia đình như tình trạng gia đình, trình độ học vấn của ba mẹ, mối quan hệ của học sinh với gia đình có phải là những yếu tố dựa đoán mạnh mẽ rằng học sinh sẽ học tập tốt hơn hay không.

**What benefits or insights would be answering this question provide?**

- Học vấn của mẹ (Medu) hay của bố (Fedu) có ảnh hưởng lớn hơn đến kết quả học tập của con cái.
    
**Who would care about the answer?**
- Giáo viên, hiệu trưởng: để hiểu rõ hoàn cảnh của học sinh và từ đó có cái nhìn khách quan hơn, và có các phương pháp tiếp cận phù hợp.
- Bộ giáo dục: có các chính sách để bổ trợ cho các nhóm học sinh yếu thế.

**What real-world problem or decision does this inform?** 

Kết quả phân tích giúp ta xây dựng được các chương trình **Can thiệp sớm**. Thay vì đợi học sinh học yếu đi thì mới hỗ trợ hay chỉ hỗ trợ những học sinh nghèo vượt khó. Ta còn có thể dựa vào các yếu tố về gia đình có thể xác định các học sinh vào nhóm có *nguy cơ cao* ngay từ đầu năm học để cung cấp và hỗ trợ về cố vấn học tập hay tài chính kịp thời.

### **A. Preprocessing**
Các bước preprocessing:

1. **Data Preparation**: Trích xuất các đặc trưng có liên quan đến gia đình của học sinh $\rightarrow$ dùng để phân tích

2. **Feature Engineering**: Tạo đặc trưng `parent_edu`: Gồm trung cộng của `Medu` và `Fedu`
    
3. **Data Cleaning**: Gom nhóm dữ liệu `Medu` và `Fedu` do `Medu = 0` và `Fedu = 0` có số lượng quá ít (không có giá trị thực tiễn) $\rightarrow$ gom nhóm dữ liệu với ` Fedu || Medu = 1`

In [ ]:
df = pd.read_csv("../data/processed/student-clean.csv")

In [ ]:
# Chuẩn bị data cho câu hỏi liên quan
fam_features = ["famsize", "Pstatus", "Medu", "Fedu", "Mjob", "Fjob", "famsup", "famrel", "parent_edu"]

ordinal_features = ["Medu", "Fedu", "famrel", "parent_edu"]

cat_features = [f for f in fam_features if f not in ordinal_features]

# Tạo dataframe chỉ gồm các yếu tố liên quan đến gia đình
df_fam = df[[f"{f}" for f in fam_features]].copy()

# Thêm cột target và dataframe
df_fam['G3'] = df['G3']

In [ ]:
# Gộp các giá trị Medu và Fedu = 0 vào mức 1 
df_fam['Medu'] = df_fam['Medu'].replace(0, 1)
df_fam['Fedu'] = df_fam['Fedu'].replace(0, 1)

# Do đã cập nhật nên sẽ liên quan đến biến parent_edu
df_fam['parent_edu'] = (df_fam['Medu'] + df_fam['Fedu']) / 2

### **B. Analysis**

1. **Visualization**: sử dụng **Boxplot** vẽ các cột categorical (so sánh sự phân tán của các cột). Đối với các cột Numerical sẽ vẽ thêm đường trung bình trong **Boxplot**. Và sau cùng là vẽ **Heatmap** để so sánh độ tương quan tổng thể của các biến Numerical (bao gồm cả biến target)

2. **Kiểm định thống kê**: để xác thực các quan sát rút ra từ biểu đồ, sử dụng kiểm định **T-test** (cho các cột có biến nhị phân) và **ANOVA** (cho các cột có nhiều biến) với ngưỡng tin cậy là 95%

Boxplot cho các biến categorical

In [ ]:
def plot_cat_grid(df, cat_cols, target = 'G3', n_cols = 3):
    """
    Vẽ grid các boxplot cho các biến categorical
    """
    
    n_vars = len(cat_cols)
    n_rows = math.ceil(n_vars / n_cols)
    
    fig, axes  = plt.subplots(n_rows, n_cols, figsize= (n_cols*5, n_rows*5))
    
    axes = axes.flatten()
    
    for i, col in enumerate(cat_cols):
        
        sns.boxplot(x = col, y = target, data= df, palette="Set2", hue=col, legend=False, ax = axes[i])
        
        axes[i].set_title(f"Tác động của {col} đến biến {target}")
        axes[i].set_xlabel('')
        axes[i].set_ylabel(target)
        
    for j in range(i+1, len(axes)):
        fig.delaxes(axes[j])
            
    plt.tight_layout()
    plt.show()

In [ ]:
plot_cat_grid(df_fam, cat_features)

Vẽ Line Plot cho các biến Numerical

In [ ]:
def plot_line_grid(df, ordinal_cols, target = 'G3', n_cols = 3):      
    n_vars = len(ordinal_cols)
    n_rows = math.ceil(n_vars / n_cols)
    
    fig, axes  = plt.subplots(n_rows, n_cols, figsize= (n_cols*5, n_rows*5))
    
    axes = axes.flatten()
    
    for i, col in enumerate(ordinal_cols):
        sns.lineplot(x=col, y=target, data=df, ax=axes[i], 
                     marker='o', color='red', linewidth=2.5)
        
        axes[i].set_title(f"Tác động của {col}", fontsize=12)
        axes[i].set_xlabel(col, fontsize=10)
        axes[i].set_ylabel("G3")
        
        axes[i].grid(True, linestyle='--', alpha=0.7)
        
    for j in range(i+1, len(axes)):
        fig.delaxes(axes[j])
    
    plt.tight_layout()
    plt.show()

In [ ]:
plot_line_grid(df_fam, ordinal_features, n_cols=2)

#### **Thực hiện lại bằng T-test/ANOVA**

In [ ]:
def statistical_testing(df, cat_col, target = 'G3'):
    """
    Hàm kiểm tra dùng T-test hoặc ANOVA dự trên số nhóm
    """
    print("-" * 20)
    groups = df.groupby(cat_col)[target].apply(list)
    
    if len(groups) == 2:
        stat, p_value = stats.ttest_ind(groups.iloc[0], groups.iloc[1])
        test_type = "T-test"
        
    else:
        stat, p_value = stats.f_oneway(*groups)
        test_type = "ANOVA"
        
    print(f"\nKiểm định cột {cat_col}")
    print(f"Loại test: {test_type}")
    print(f"P-value: {p_value:3f}")
        
    if p_value < 0.05:
        print(f"Có sự ảnh hưởng (Sự khác biệt có ý nghĩa thống kê)\n")
        return 1
    else:
        print(f"Không ảnh hưởng (Sự khác biệt chỉ là ngẫu nhiên)\n")
        return 0

In [ ]:
# Kiểm tra các biến bằng T-test và ANOVA
affect_features = []
for cat in fam_features:
    if(statistical_testing(df_fam, cat)):
        affect_features.append(cat)

print(affect_features)

### **C. Results & Interpretation**

#### **Kết quả nghiên cứu**  

Dựa vào các kết quả phân tích trực quan và kiểm định thống kê, ta có thể khẳng định rằng trong tập dataset này các yếu tố gia đình có ảnh hưởng đáng kể đến kết quả học tập của học sinh. Nhưng các yếu tố ảnh hưởng không đồng đều:  

- Ảnh hưởng mạnh nhất: Nhóm về nền tảng tri thức `parent_edu` (hay `Medu` và `Fedu`) và nghề nghiệp (`Fedu`, `Medu`). 

- Ảnh hưởng tầm trung: Quy mô của gia đình (`famsize`).  

- Các cột còn lại không có ảnh hưởng đáng kể đến kết quả điểm số của học sinh.  

#### **Phân tích chi tiết**  

**1. Tác động của nền tảng tri thức (`parent_edu`)**  
Dựa vào biểu đồ Boxplot và kiểm định ANOVA (p_value = 0.05) ta thấy có 1 xu hướng rõ rệt là, trình độ học vấn của cha mẹ càng cao thì điểm trung bình của con họ càng cao theo.

- Dẫn chứng: Ta thấy khi `parent_edu = 3.5 hay 4.0` thì điểm trung vị đạt khoảng 12.5 điểm, cao hơn đáng kể so với nhóm học sinh có cha mẹ học thấp hơn.

- Ý nghĩa: Điều này cho ta thấy nền tảng tri thức có ảnh hưởng rất nhiều đến nhận thức và là nền tảng cho học sinh có khả năng và phương pháp học tập tốt hơn.

**2. Tác động của nghề nghiệp (`Mjob`, `Fjob`)**  
Nghề nghiệp còn phản ánh môi trường tri thức, thói quen tại gia đình

- Dẫn chứng: Các học sinh có bậc cha mẹ làm giáo viên (`teacher`) hoặc làm trong khối ngành sức khoẻ (`health`) có mức điểm vượt trội hơn các nhóm còn lại.

- Ý nghĩa: Sự chênh lệch này gợi ý rằng sự ổn định về kinh tế và môi trường khuyến học tạo ra lợi thế cạnh tranh lớn cho học sinh.

**3. Tác động quy mô gia đình (`famsize`)**   
Mặc dù nhìn vào Boxplot thấy sự chênh lệch các giá trị không quá lớn, nhưng sau khi kiểm định T-test đã xác định được sự khác biệt về mặc thống kê

- Dẫn chứng: Học sinh có gia đình quy mô nhỏ thì điểm sẽ ổn định hơn các gia đình có quy mô lớn.

- Ý nghĩa: Điều này rất phù hợp với giả thuyết rằng ở một gia đình ít con hơn thì vấn đề tiền bạc hay sự quan tâm đều chất lượng hơn nhiều so với gia đình có đông con (bị chia nhỏ ra nên ít có sự quan tâm hơn)

#### **Những phát hiện trong phân tích dữ liệu**

**Nghịch lý trong `famsup`**  
Một điều trái với những suy đoán thông thường (nếu được gia đình ủng hộ nhiều trong việc học tập thì điểm số của học sinh sẽ tốt hơn) nhưng ở đây dù cho có được ủng hộ hay không thì cũng không tạo ra được sự khác biệt về điểm số của 2 nhóm đó.

- Phân tích: Cả 2 nhóm "Được hỗ trợ" và "Không được hỗ trợ" đều có trung vị tương đương với nhau.

- Ý nghĩa: Việc nhóm học sinh "Không được hỗ trợ" có trung vị ngang với nhóm còn lại chứng tỏ việc các em học sinh nhận thức được việc cần phải tự cố gắng học tập cho nên điểm trung vị của 2 bên là xấp xỉ nhau.

#### **Limitations**
- Tương quan không phải Nhân quả: Việc con giáo viên học giỏi có thể do gen di truyền hoặc môi trường, không chắc chắn 100% là do nghề nghiệp quyết định.


## **Q2: Dựa vào các thói quen của học sinh có thể dự đoán họ có trở nên uống rượu nhiều hơn hay không? (sử dụng Machine Learning)**

### **The question**
- Dữ liệu sử dụng sẽ tập trung vào các biến hành vi (thói quen) của học sinh như `goout`, `freetime`,... để sự đoán về biến mục tiêu `total_alc` rằng học sinh đó sẽ được phân loại vô nhóm uống nhiều hay uống ít.

### **Motivation and Benefits**
**Why is this question worth investigating?**  

Ta tập trung vào những yếu tố có thể khắc phục, trái với các yếu tố gia đình thì thói quen ta có thể can thiệp và điều chỉnh sao cho hợp lí. Và một phần cũng giúp kiểm chứng được tác động của những thói quen đó có làm cho một người uống rượu nhiều hơn hay không.

**What benefits or insights would be answering this question provide?**
- Xác định được những thói quen độc hại nào sẽ có nguy cơ gây uống rượu cao ở học sinh.
- Dự báo sớm dự trên các hành vi, chỉ cần quan sát trong quá trình sinh hoạt, thói quen có thể dự báo được nguy cơ.

**Who would care about the answer?**
- Phụ huynh: Để họ hiểu rằng các thói quen sinh hoạt cũng rất quan trọng chứ không chỉ là có điểm số.
- Giáo viên, tư vấn viên: Họ có cơ sở để phát hiện các học sinh có nguy cơ uống rượu cao và từ đó có các cách can thiệp sớm kịp thời.

**What real-world problem or decision does this inform?**  

Giúp nhà trường, phụ huynh phát hiện sớm hành vi để có thể can thiệp kịp thời, tránh những hành vi sa ngã quá mức. Ngoài ra còn có thể thiết kế các chương trình giáo dục kĩ năng, giúp điều chỉnh thói quen hoặc nâng cao nhận thức của học sinh ngay từ sớm để ngăn chặn nguy cơ.

### **A. Preprocessing**
Các bước preprocessing:
- Trích xuất các feature về thói quen của học sinh gồm (`studytime`, `activities`, `internet`, `romantic`, `freetime`, `goout`, `total_alc`)

- Encoding: các biến binary categorical (`yes = 1` và `no = 0`)

- Feature Engineering: biến target `is_high_risk` nếu `total_alc > 3` $\rightarrow$ `is_high_risk = 1`


In [ ]:
df = pd.read_csv("../data/processed/student-clean.csv")

In [ ]:
habit_features = ["studytime", "activities", "internet", "romantic", "freetime", "goout"]

df_habit = df[habit_features].copy()
df_habit['total_alc'] = df['total_alc'].copy()

In [ ]:
# Encoding các biến categorical
mapping_binary = {
    'yes' : 1,
    'no' : 0,
}

binary_features = ["activities", "internet", "romantic"]
for f in binary_features:
    df_habit[f] = df_habit[f].map(mapping_binary)

### **B. Analysis**

- **Colleration heatmap**: Xem có mối liên quan giữa các biến thói quen với biến target (`total_alc`), biến nào ảnh hưởng nhiều, ảnh hưởng ít đến `total_alc` $\rightarrow$ chú ý đặc biệt hơn đến những điều đó.

- **Kiểm tra độ cân bằng của target**: Xem các giá trị của biến target có bị mất cân bằng hay không $\rightarrow$ có hướng đi phù hợp để xử lý cân bằng.

- **Lựa chọn mô hình**:
    - Logistic Regression: Có khả năng phân tích các biến (do có cung cấp hệ số `Coeficients` cho từng biến); Phù hợp với dữ liệu nhỏ ít bị Overfitting nhanh gọn và hiệu quả.
    
    - Random Forest: Có khả năng nắm bắt quan hệ phi tuyến tính phức tạp; Đây là mô hình tổ hợp nhiều câu quyết định nên thường có độ chính xác cao và ổn định hơn; Có thể tính độ quan trọng của biến (Feature Importance).

- **Metric Evaluation**: vì biến Target bị mất cân bằng, nên thay vì dùng Accuracy ta sẽ so sánh Recall và Precision của các mô hình để tìm ra mô hình tối ưu.

#### **Vẽ Colleration Heatmap của các biến**

In [ ]:
def plot_correlation_heatmap(df, num_cols):
    """
    Vẽ heatmap các biến Numerical (Ordinal) 
    """
    
    corr_matrix = df[num_cols].corr(method='spearman')
    
    plt.figure(figsize=(12,10))
    
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    
    sns.heatmap(corr_matrix,
                mask= mask,
                annot = True,
                fmt = ".2f",
                cmap = "coolwarm",
                vmin=-1, vmax=1,
                center=0,
                linewidths=0.5,
                cbar_kws={"shrink" : .8})
    
    plt.title(f"Biểu đồ tương quan của các biến cột Numerical (Ordinal)")
    plt.show()

In [ ]:
habit_target_features = habit_features + ["total_alc"]

plot_correlation_heatmap(df_habit, habit_target_features)

**Nhận xét:** 
- Nhóm có độ tương quan cao với biến `total_alc`: `goout` (tương quan thuận), `studytime` (tương quan nghịch) $\rightarrow$ Điều đó cho thấy việc càng dành nhiều thời gian để đi chơi sẽ khiến học sinh càng uống rượu nhiều hơn và với việc dành nhiều thời gian để học tương đương với việc uống ít rượu hơn.

- Các nhóm còn lại có tác động không đáng kể.

#### **Kiểm tra biến Target**

In [ ]:
# Tạo cột binary cateogircal với ngưỡng > 3 -> uống nhiều
df_habit['is_high_risk'] = (df_habit['total_alc'] > 3).astype(int)

# Visualization
plt.figure(figsize=(6,4))

sns.countplot(data = df_habit, x = 'is_high_risk', palette='pastel', hue = 'is_high_risk', legend=False)

plt.title("Phân phối của biến target")
plt.xlabel("Tình trạng nguy hiểm (0 = Low, 1 = High)")
plt.ylabel("Số lượng học sinh")
plt.show()

**Nhận xét**: Biến target bị mất cân bằng dữ liệu khá nhiều

#### **Sử dụng Machine Learning để dự đoán**

In [ ]:
# Chuẩn bị dữ liệu train
target_col = 'is_high_risk'

X = df_habit[habit_features].copy()
y = df_habit[target_col].copy()

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 33)

In [ ]:
scaler = StandardScaler()

# Scale lại input
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
def evalutate_model(model_name, y_true, y_pred):
    print(f"Model: {model_name}")
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.2f}")
    print(f"Classification report: \n{classification_report(y_true, y_pred)}")

In [ ]:
def confusion_matrix_plot(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)

    # Vẽ Heatmap
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, 
                xticklabels=['Low Risk (0)', 'High Risk (1)'],
                yticklabels=['Low Risk (0)', 'High Risk (1)'])

    plt.xlabel('Predicted Label')
    plt.ylabel('Actual Label')
    plt.title('Confusion Matrix')
    plt.show()

**Sử dụng Logistic Regression**

In [ ]:
# Logistic Regression với class_weight
lr_model = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=33)

lr_model.fit(X_train_scaled, y_train)
y_pred_lr = lr_model.predict(X_test_scaled)

evalutate_model("Logistis Regression with class_weight", y_test, y_pred_lr)

In [ ]:
confusion_matrix_plot(y_test, y_pred_lr)

**Sử dụng Random Forest**

In [ ]:
# RandomForest với clas_weight
rf_model = RandomForestClassifier(class_weight='balanced', random_state=33, n_estimators=100)

rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

evalutate_model("Random Forest with class_weight", y_test, y_pred_rf)

In [ ]:
confusion_matrix_plot(y_test, y_pred_rf)

#### **Feature Interpretation**

In [ ]:
feature_names = X_train.columns

# Lấy hệ số (coefficients) từ model Logistic Regression
coefficients = lr_model.coef_[0]

feature_importance = pd.DataFrame({
    'Habit': feature_names,
    'Coefficient': coefficients,
    'Absolute_Impact': abs(coefficients) # Lấy trị tuyệt đối 
})

# Sắp xếp theo mức độ ảnh hưởng giảm dần
feature_importance = feature_importance.sort_values(by='Absolute_Impact', ascending=False)
print(feature_importance)

plt.figure(figsize=(10, 6))
sns.barplot(x='Coefficient', y='Habit', data=feature_importance, palette='pastel', hue = 'Habit', legend=False)
plt.title('Mức độ ảnh hưởng của các biến đến Target')
plt.axvline(x=0, color='black', linestyle='--')
plt.show()

### **C. Results & Interpretation**

#### **Model performance & Selection**
Sau khi thực hiện 2 mô hình, ta có kết quả như sau:
- Logistic Regression: Recall (62%) đối với nhóm `is_high_risk = 1` và Precision (15%)
- Random Forest: Recall (38%) đối với nhóm `is_high_risk = 1` và Precision (17%)

$\Rightarrow$ Quyết định lựa chọn **Logistic Regression** làm mô hình cuối cùng. Vì bài toán đặt ra là phát hiện rủi ro nếu một học sinh có nguy cơ uống rượu nhiề hay không (liên quan nhiều đến Recall) cho nên điểm Recall cao sẽ được ưu tiên nhiều hơn.

#### **Feature Interpretation**
Dựa trên các hệ số hồi quy đã được chuẩn hoá trong mô hình Logistic Regression, ta thấy:
- Yếu tố thúc đẩy: `goout` có hệ sống dương cao nhất $\rightarrow$ Khẳng định tần suất tham gia các hoạt động vui chơi bên ngoài là chỉ báo rủi ro cao.
- Yếu tố kìm hãm: `studytime` có hệ số âm sâu nhất $\rightarrow$ Thời gian dành cho việc học tỉ lệ nghịch với khả năng uống nhiều rượu bia.

#### **Kết luận**
Dữ liệu cho thấy hành vi uống rượu không chịu ảnh hửng nhiều bảo các yếu tố tác động bên ngoài mà là kết quả của sự cạnh tranh quỹ thời gian. Ta thấy khi học sinh càng dành nhiều thời gian học `studytime` hơn đồng nghĩa với việc thời gian rảnh cho việc đi chơi `goout` bị giảm $\rightarrow$ Giảm thiểu cơ hội tiếp xúc với rượu bia. Cho nên nếu muốn ngăn chặn cần phải tập trung điều chỉnh cân bằng lối sống thay vì cấm đoán trực tiếp.